In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2
conn = psycopg2.connect(dbname='Yellow-Taxi', user='postgres', password='7466239', host='localhost', port='5432')
# Open a cursor to perform database operations
cur = conn.cursor()

In [2]:
pd.read_sql_query('SELECT * FROM taxi_data limit 2', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_14136\2611953238.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('SELECT * FROM taxi_data limit 2', conn)


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,rounded_date
0,2,2023-11-21 11:30:54,2023-11-21 11:59:53,1,5.1,1,N,140,148,1,32.4,0.0,0.5,7.28,0.0,1.0,43.68,2.5,0.0,2023-11-21 11:00:00
1,1,2023-11-21 11:09:53,2023-11-21 11:14:15,1,0.7,1,N,246,246,1,6.5,2.5,0.5,0.00,0.0,1.0,10.50,2.5,0.0,2023-11-21 11:00:00


In [5]:
pd.read_sql_query('''
select distinct payment_type
from taxi_data
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_14136\189143594.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,payment_type
0,0
1,1
2,2
3,3
4,4
5,5


### repair anomalies in raw data

In [14]:
pd.read_sql_query('''
select
max(fare_amount) as max_fare,
min(fare_amount) as min_fare,
max(tip_amount) as max_tip,
min(tip_amount) as min_tip
from taxi_data
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_12500\1134658114.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,max_fare,min_fare,max_tip,min_tip
0,401092.32,-133391414.0,4174.0,-411.0


In [2]:
cur.execute('''
UPDATE taxi_data
SET tip_amount = 0
WHERE tip_amount > 10000 ''')
conn.commit()

### create the summary table for gcp

In [3]:
cur.execute(''' drop table if exists taxi_data_summary ''')
conn.commit()

In [4]:
cur.execute('''
create table taxi_data_summary as
with taxi_db as (
select rounded_date,
count(*) as num_trips,
sum(passenger_count) as total_passengers,
sum(trip_distance) as total_distance,
sum(case when store_and_fwd_flag = 'Y' then 1 else 0 end) as num_store_and_fwd,
sum(case when payment_type = 0 then 1 else 0 end) as num_payment_type_0,
sum(case when payment_type = 1 then 1 else 0 end) as num_payment_type_1,
sum(case when payment_type = 2 then 1 else 0 end) as num_payment_type_2,
sum(case when payment_type = 3 then 1 else 0 end) as num_payment_type_3,
sum(case when payment_type = 4 then 1 else 0 end) as num_payment_type_4,
sum(case when payment_type = 5 then 1 else 0 end) as num_payment_type_5,
sum(fare_amount) as total_fare,
sum(case when extra > 0 then 1 else 0 end) as num_extra,
sum(extra) as total_extra,
sum(mta_tax) as total_mta_tax,
sum(case when tip_amount > 0 then 1 else 0 end) as num_tips,
sum(tip_amount) as total_tips,
sum(case when tolls_amount > 0 then 1 else 0 end) as num_tolls,
sum(tolls_amount) as total_tolls,
sum(case when improvement_surcharge > 0 then 1 else 0 end) as num_improvement_surcharge,
sum(improvement_surcharge) as total_improvement_surcharge,
sum(total_amount) as total_amount,
sum(case when congestion_surcharge > 0 then 1 else 0 end) as num_congestion_surcharge,
sum(congestion_surcharge) as total_congestion_surcharge,
sum(case when airport_fee > 0 then 1 else 0 end) as num_airport_fee,
sum(airport_fee) as total_airport_fee
from taxi_data
where EXTRACT(YEAR FROM rounded_date) IN (2022, 2023)
group by rounded_date
),
climate_db as (
select
rounded_date,
avg(temperature) AS avg_temperature,
avg(wind_speed) AS avg_wind_speed,
avg(precipitation) AS avg_precipitation,
avg(relative_humidity) AS avg_relative_humidity,
avg(visibility) AS avg_visibility
FROM climate
WHERE temperature != 'NaN' AND wind_speed != 'NaN' AND relative_humidity != 'NaN' AND visibility != 'NaN'
GROUP BY 1
)
select t1.*, t2.avg_temperature, t2.avg_wind_speed, t2.avg_precipitation, t2.avg_relative_humidity, t2.avg_visibility
from taxi_db t1
left join climate_db t2
on t1.rounded_date = t2.rounded_date
''')
conn.commit()

### Tests and Checks

In [50]:
pd.read_sql_query('''
SELECT rounded_date, count(*) as freq
FROM taxi_data_summary
group by 1
order by 2 desc
limit 10
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_18540\2177588762.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,rounded_date,freq
0,2022-09-21 04:00:00,1
1,2022-09-08 21:00:00,1
2,2023-11-19 00:00:00,1
3,2022-07-27 16:00:00,1
4,2023-10-17 11:00:00,1
5,2023-02-28 07:00:00,1
6,2022-07-29 04:00:00,1
7,2022-06-04 23:00:00,1
8,2023-03-18 19:00:00,1
9,2022-11-29 21:00:00,1


In [31]:
pd.read_sql_query('''
SELECT table_name FROM information_schema.tables where table_schema = 'public'
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_18540\1608219410.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,table_name
0,taxi_data
1,climate
2,taxi_data_summary


In [36]:
pd.read_sql_query('''SELECT min(rounded_date), max(rounded_date) FROM taxi_data_summary limit 2''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_18540\3501630287.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''SELECT min(rounded_date), max(rounded_date) FROM taxi_data_summary limit 2''', conn)


,min,max
0,2022-01-01,2023-12-31 23:00:00


## export summary table to gcp

In [15]:
import pandas as pd
from google.cloud import storage
import sqlite3

# Load the data into a DataFrame
df = pd.read_sql(' SELECT * FROM taxi_data_summary ', conn)

# Save the DataFrame to a CSV file
df.to_csv('taxi_data_summary.csv', index=False)

# Upload the CSV file to GCP
storage_client = storage.Client()
bucket = storage_client.bucket("yellow_taxi_climate")
blob = bucket.blob('taxi_data_summary.csv')
# Delete the existing file with the same name
blob.delete()
blob.upload_from_filename('taxi_data_summary.csv')

print('File uploaded successfully ')

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_12500\3965207996.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(' SELECT * FROM taxi_data_summary ', conn)


File uploaded successfully 


In [40]:
pd.read_sql_query('''
SELECT COUNT(*) FROM information_schema.columns WHERE table_name = 'taxi_data_summary'
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_18540\4158836666.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,count
0,31


### overwrite the gcp table

In [16]:
# updating the table in BigQuery:
from google.cloud import bigquery
from google.cloud import storage

client = bigquery.Client()
table_id = "footbal-climate.Yellow_Taxi.yellow_taxi_main"

# Get the blob from GCS
storage_client = storage.Client()
storage_client.bucket('yellow_taxi_climate').blob('taxi_data_summary.csv').download_to_filename('/tmp/taxi_data_summary.csv')


job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)
with open('/tmp/taxi_data_summary.csv', 'rb') as source_file:
    load_job = client.load_table_from_file(source_file, table_id, job_config=job_config)

load_job.result()
print("Table overwritten successfully.")


Table overwritten successfully.
